# Loops through prediction xml files and images to score them.


In [10]:
print("Keyboard commands")
print("This process requires keyboard input while an image is displayed.")
print("~ quit")
print("/ Write the file xml annotation file and move to next image")
print("| object is present in the image")
print("_ object is not present in the image")
print("+ Go forward 1 images")
print("> Go forward 9 images")
print("? Go forward 99 images")
print("- Go back 1 image")
print("a - z     toggle boxes 1  - 26")
print("0 - 9     toggle boxes 27 - 36")
print("space - ) toggle boxes 37 - 46")

print("Continuation of keyboard commands to toggle if predictions are correct or not.")
for cn in range(1,27):
    print("box ", cn, " press", chr(cn+96))
for cn in range(0,10):
    print("box ", cn+27, " press", chr(cn+48))    
for cn in range(0,10):
    print("box ", cn+37, " press", chr(cn+32))

Keyboard commands
This process requires keyboard input while an image is displayed.
~ quit
/ Write the file xml annotation file and move to next image
| object is present in the image
_ object is not present in the image
+ Go forward 1 images
> Go forward 9 images
? Go forward 99 images
- Go back 1 image
a - z     toggle boxes 1  - 26
0 - 9     toggle boxes 27 - 36
space - ) toggle boxes 37 - 46
Continuation of keyboard commands to toggle if predictions are correct or not.
box  1  press a
box  2  press b
box  3  press c
box  4  press d
box  5  press e
box  6  press f
box  7  press g
box  8  press h
box  9  press i
box  10  press j
box  11  press k
box  12  press l
box  13  press m
box  14  press n
box  15  press o
box  16  press p
box  17  press q
box  18  press r
box  19  press s
box  20  press t
box  21  press u
box  22  press v
box  23  press w
box  24  press x
box  25  press y
box  26  press z
box  27  press 0
box  28  press 1
box  29  press 2
box  30  press 3
box  31  press 4
box 

In [13]:
import os
from os import listdir
import cv2
import pickle
import cv2
import skimage
import random
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset
from mrcnn.visualize import display_instances
from mrcnn.utils import extract_bboxes

object_type = "charcoal_hearth_hill"
cfg_name = 'cfg20200826T2315'
model_epoch='0016'

print("cfg_name:",cfg_name)

# The annotations made for predictions on the 100 test images
image_prediction_folder = os.path.join('/home/student/', object_type, 'predictions/' + (cfg_name + '/'), 'unknown_training/')

image_prediction_folder_edited = os.path.join(image_prediction_folder,'edited/')
if not os.path.exists(image_prediction_folder_edited):
    os.mkdir(image_prediction_folder_edited)
    

# image_for_prediction_folder = '/home/student/' + object_type + '/images/jpgs/'
image_for_prediction_folder = '/home/student/' + object_type + '/images_training/'

# These are the original training annot files.  They will be displayed in white
training_annots_folder = '/home/student/' + object_type + '/annots/'

# This is to test 100 random images
# with open('rcnn_100_random_images_charcoal_hearth_hill.pkl', 'rb') as f:
with open('rcnn_100_random_images_training_charcoal_hearth_hill.pkl', 'rb') as f:
    test_file_names = pickle.load(f)

#This is to test all of the files 
#test_file_names = []
#for filename in listdir(image_for_prediction_folder):
    # print(filename)
#    test_file_names.append(filename)

# display image with masks and bounding boxes


# set up font for boxes
# https://www.geeksforgeeks.org/python-opencv-cv2-puttext-method/
# font 
font = cv2.FONT_HERSHEY_SIMPLEX 
font_scale = 1
font_color = (212, 212, 0) 
font_thickness = 2
   
cn=0
while cn<len(test_file_names):
    
    print("count ",cn)
    image = skimage.io.imread(os.path.join(image_for_prediction_folder, test_file_names[cn]))
    print(image.shape)
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    #https://stackoverflow.com/questions/32609098/how-to-fast-change-image-brightness-with-python-opencv
    #make it brighter
    
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    v += 42
    final_hsv = cv2.merge((h, s, v))

    image = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)

    obj_f_num = test_file_names[cn]
    obj_f_num = obj_f_num[:10]
    print(cn,obj_f_num)
    overlay = image.copy()
    output = image.copy()
    
    #Get original training data too
    # Most of the random predictions won't have training data.
    training_annot_file_path = os.path.join(training_annots_folder, (obj_f_num+'.xml'))
    if(os.path.exists(training_annot_file_path)):
        
        tree_train = ElementTree.parse(training_annot_file_path)
        print(training_annots_folder+obj_f_num+'.xml')
        #print(tree_train)
        # get the root of the document
        root_train = tree_train.getroot()
        for obj in root_train.findall('./object'):
            box = obj.find('bndbox')
            xmin = int(box.find('xmin').text)
            ymin = int(box.find('ymin').text)
            xmax = int(box.find('xmax').text)
            ymax = int(box.find('ymax').text)
            if(ymin>ymax):
                ytemp = ymin
                ymin = ymax
                ymax=ytemp
            if(xmin>xmax):
                xtemp = xmin
                xmin = xmax
                xmax=xtemp            
            coors = [xmin, ymin, xmax, ymax]
            #print(coors)

            cv2.rectangle(output, (xmin, ymin), (xmax, ymax),(255, 255, 255), 6)
    else:
        print("No training annotation file for this image.  This is likely ok.")
    
    cv2.putText(output, str(cn), (10,30), font, font_scale, font_color, font_thickness, cv2.LINE_AA)
    
    # Look for the edited file first.  Otherwise, get the original prediction
    # Look for the edited file first.  Otherwise, get the original prediction
    annot_path = os.path.join(image_prediction_folder, "edited/", (obj_f_num+'.xml'))
    if(os.path.exists(annot_path)):
        print("Using edited version.")
    else:
        annot_path = os.path.join(image_prediction_folder, (obj_f_num+'.xml'))
    
    tree = ElementTree.parse(annot_path)
    print(image_prediction_folder+obj_f_num+'.xml')
    #print(tree)
    # get the root of the document
    root = tree.getroot()
    # extract each bounding box
    
    fn_image = root.find('./filename').text
    object_present = root.find('./object_present').text
    #    object_present="0"
    #    try:
    #        object_present=root.find('./object_present').text
    #    except:
    #        print("element object_present not found")
    cv2.putText(output, str(object_present), (70,30), font, font_scale/2, font_color, font_thickness, cv2.LINE_AA)
    #print(fn_image)
    box_num=0

    boxes_correct={}
    for obj in root.findall('./object'):
        score = obj.find('score').text
        correct = obj.find('correct').text
        if(correct is None):
            correct="0"
        if(correct==""):
            correct="0"
        correct=int(correct)
        box = obj.find('bndbox')
        box_num=obj.find('number').text
        boxes_correct[str(box_num)] = correct
        xmin = int(box.find('xmin').text)
        ymin = int(box.find('ymin').text)
        xmax = int(box.find('xmax').text)
        ymax = int(box.find('ymax').text)
        if(ymin>ymax):
            ytemp = ymin
            ymin = ymax
            ymax=ytemp
        if(xmin>xmax):
            xtemp = xmin
            xmin = xmax
            xmax=xtemp            
        coors = [xmin, ymin, xmax, ymax]
        #print(coors)
        if(correct==0):
            cv2.rectangle(output, (xmin, ymin), (xmax, ymax),(0, 0, 255), 2)
        elif(correct==1):
            cv2.rectangle(output, (xmin, ymin), (xmax, ymax),(0, 255,0 ), 2)
        # Using cv2.putText() method 
        #cv2.putText(output, '['+box_num+'] S: '+ score[:5], (xmin+10, ymin+70), font, font_scale, font_color, font_thickness, cv2.LINE_AA) 
        
        if (int(box_num)<=25):
            box_keys=chr(65+32+int(box_num))
        if (int(box_num)>=26 and int(box_num)<=35):
            box_keys=chr(48+int(box_num)-26)
        if (int(box_num)>=36 and int(box_num)<=45):
            box_keys=chr(32+int(box_num)-36)
            
        if (ymin+80)<image.shape[0]:
            cv2.putText(output, box_keys , (xmin+10, ymin+70), font, font_scale, font_color, font_thickness, cv2.LINE_AA) 
        else:
            cv2.putText(output, box_keys , (xmin+10, ymin-10), font, font_scale, font_color, font_thickness, cv2.LINE_AA) 
        
    alpha = .8
    #cv2.addWeighted(overlay, alpha, output, 1-alpha,0, output)
    #cv2.putText(output, '['+str(cn)+'] Present: '+ object_present, (50, 70), font, font_scale, font_color, font_thickness, cv2.LINE_AA) 
    image = output
    scale_percent = 1024/image.shape[1] # percent of original size
    #print(scale_percent)
    width = int(image.shape[1] * scale_percent )
    height = int(image.shape[0] * scale_percent )
    dim = (width, height)
    # resize image
    resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    resized_original = resized.copy()
    
    object_file_num_int_return = -1
 # keep looping until the 'c' key is pressed
    while True:
        # display the image and wait for a keypress
        cv2.imshow("image", resized)
        cv2.moveWindow("image", 10,10)
        #key = cv2.waitKey(1) & 0xFF
        key = cv2.waitKey()

        # if the '!' key is pressed, break from the loop
        #note if object is found anywhere in the image or not
        if key == ord("|"):
            object_present="1"
        if key == ord("_"):
            object_present="0"
        if key == ord("/"):
            object_file_num_int_return = -1
            break
        if key == ord("+"):
            break
        if key == ord("-"):
            cn = cn-2
            break
        if key == ord(">"):
            cn = cn+9
            break
        if key == ord("?"):
            cn = cn+99
            break
           
        if key == ord("~"):
            break
        if key>=ord("a") and key<=ord("z"):
            bc_key = key-97
            if(bc_key<len(boxes_correct)):
                print("                                                           ","low key ord",key,bc_key,boxes_correct[str(bc_key)],str(int(not int(boxes_correct[str(bc_key)]))))
                boxes_correct[str(bc_key)]=str(int(not int(boxes_correct[str(bc_key)])))

        if key>=ord("0") and key<=ord("9"):
            bc_key = key-48+26
            if(bc_key<len(boxes_correct)):
                print("                                                           ","key ord",key,key-48,bc_key,boxes_correct[str(bc_key)],str(int(not int(boxes_correct[str(bc_key)]))))
                boxes_correct[str(bc_key)]=str(int(not int(boxes_correct[str(bc_key)])))

        if key>=ord(" ") and key<=ord(")"):
            bc_key = key-32+36
            if(bc_key<len(boxes_correct)):
                print("                                                           ","key ord",key,key-32,bc_key,boxes_correct[str(bc_key)],str(int(not int(boxes_correct[str(bc_key)]))))
                boxes_correct[str(bc_key)] = str(int(not int(boxes_correct[str(bc_key)])))
        # set all correct
        if key==ord("]"):
            print("all correct")
            for box_cn in range(0,len(boxes_correct)):
                boxes_correct[str(box_cn)] = "1"
            # boxes_correct[1] = "1"
        # set all not correct
        if key==ord("["):
            for box_cn in range(0,len(boxes_correct)):
                boxes_correct[str(box_cn)] = "0"
        print(chr(key))
                

    if object_file_num_int_return == -1:
        #correct_number = root.find('./correct_number')
        for obj in root.findall('./object'):
            box_num=obj.find('number').text
            correct = obj.find('correct')
            correct.text = str(boxes_correct[str(box_num)])
            #print("                                                           ",box_num, boxes_correct[str(box_num)],correct.text)
            #print(boxes_correct[str(box_num)])
        #correct_number.text = str(object_file_num_int_return)
        #root.set('correct_number',str(object_file_num_int_return))
        
        op = root.find('object_present')  #.text = "xx" #object_present
        op.text = str(object_present)
        #tree.write(image_prediction_folder+obj_f_num+'.xml')
        #write out an annot file of just the correct boxes
        tree_edited = tree
        root_edited = tree_edited.getroot()
        #object_present_element = ElementTree.Element("object_present")
        #object_present_element.text = object_present
        #root_edited.append(object_present_element)

        #This is commented out to keep both correct and not correct
        #for obj in root_edited.findall('./object'):
        #    box_num=obj.find('number').text
        #    correct = obj.find('correct')
        #    if(correct.text=="0"):
        #        print("removed",obj)
        #        root.remove(obj)
        tree_edited.write(os.path.join(image_prediction_folder_edited, (obj_f_num+'.xml')))
            
            
    # close all open windows
    cv2.destroyAllWindows()
    image=None
    if key == ord("~"):
        break
    cn=cn+1
    if(cn >= len(test_file_names)):
        cn = 0
    if(cn < 0):
        cn = 0        

cfg_name: cfg20200826T2315
count  0
(768, 1024)
0 217pas2016
/home/student/charcoal_hearth_hill/annots/217pas2016.xml
Using edited version.
/home/student/charcoal_hearth_hill/predictions/cfg20200826T2315/unknown_training/217pas2016.xml
count  1
(768, 1024)
1 073pas2609
/home/student/charcoal_hearth_hill/annots/073pas2609.xml
Using edited version.
/home/student/charcoal_hearth_hill/predictions/cfg20200826T2315/unknown_training/073pas2609.xml
count  2
(768, 1024)
2 156pas0908
/home/student/charcoal_hearth_hill/annots/156pas0908.xml
Using edited version.
/home/student/charcoal_hearth_hill/predictions/cfg20200826T2315/unknown_training/156pas0908.xml
count  3
(768, 1024)
3 058pan1508
/home/student/charcoal_hearth_hill/annots/058pan1508.xml
Using edited version.
/home/student/charcoal_hearth_hill/predictions/cfg20200826T2315/unknown_training/058pan1508.xml
count  4
(768, 1024)
4 046pas0700
/home/student/charcoal_hearth_hill/annots/046pas0700.xml
Using edited version.
/home/student/charcoal_h

count  35
(768, 1024)
35 115pan0506
/home/student/charcoal_hearth_hill/annots/115pan0506.xml
Using edited version.
/home/student/charcoal_hearth_hill/predictions/cfg20200826T2315/unknown_training/115pan0506.xml
count  36
(768, 1024)
36 041pas1109
/home/student/charcoal_hearth_hill/annots/041pas1109.xml
Using edited version.
/home/student/charcoal_hearth_hill/predictions/cfg20200826T2315/unknown_training/041pas1109.xml
count  37
(768, 1024)
37 042pas0105
/home/student/charcoal_hearth_hill/annots/042pas0105.xml
Using edited version.
/home/student/charcoal_hearth_hill/predictions/cfg20200826T2315/unknown_training/042pas0105.xml
count  38
(768, 1024)
38 156pas0801
/home/student/charcoal_hearth_hill/annots/156pas0801.xml
Using edited version.
/home/student/charcoal_hearth_hill/predictions/cfg20200826T2315/unknown_training/156pas0801.xml
count  39
(768, 1024)
39 217pan0303
/home/student/charcoal_hearth_hill/annots/217pan0303.xml
Using edited version.
/home/student/charcoal_hearth_hill/predic

count  74
(768, 1024)
74 156pas0403
/home/student/charcoal_hearth_hill/annots/156pas0403.xml
Using edited version.
/home/student/charcoal_hearth_hill/predictions/cfg20200826T2315/unknown_training/156pas0403.xml
count  75
(768, 1024)
75 051pas1116
/home/student/charcoal_hearth_hill/annots/051pas1116.xml
Using edited version.
/home/student/charcoal_hearth_hill/predictions/cfg20200826T2315/unknown_training/051pas1116.xml
count  76
(768, 1024)
76 043pas1109
/home/student/charcoal_hearth_hill/annots/043pas1109.xml
Using edited version.
/home/student/charcoal_hearth_hill/predictions/cfg20200826T2315/unknown_training/043pas1109.xml
count  77
(721, 1024)
77 225pas0804
/home/student/charcoal_hearth_hill/annots/225pas0804.xml
Using edited version.
/home/student/charcoal_hearth_hill/predictions/cfg20200826T2315/unknown_training/225pas0804.xml
count  78
(768, 1024)
78 073pas1913
/home/student/charcoal_hearth_hill/annots/073pas1913.xml
Using edited version.
/home/student/charcoal_hearth_hill/predic

# Gather the data from the xml files into results_cfg_.csv

In [4]:
cfg_name = "cfg20200826T2315"
print("cfg_name:",cfg_name)
object_type="charcoal_hearth_hill"
# sample type is either "_training" or ""
sample_type = ""
import os
from os import listdir
from xml.etree import ElementTree
import pickle

with open('rcnn_100_random_images' + sample_type + '_charcoal_hearth_hill.pkl', 'rb') as f:
    test_file_names = pickle.load(f)

image_prediction_folder = os.path.join('/home/student/', object_type, 'predictions/' + (cfg_name + '/'), ('unknown' + sample_type + '/'))
#image_prediction_folder = os.path.join('/home/student/', object_type, 'predictions/' + (cfg_name + '/'), 'unknown_training/')

image_prediction_folder_edited = os.path.join(image_prediction_folder,'edited/')


# display image with masks and bounding boxes

#results = open(os.path.join(image_prediction_folder_edited , ("results_100_random_images+sample_type"_"+cfg_name+".csv")),"w+")
results = open(os.path.join(image_prediction_folder_edited , ("results_100_random_images" + sample_type + "_"+cfg_name+".csv")),"w+")
results.write("min_score,gt_obj_pres_pos,gt_obj_pres_neg,tp,fp,tn,fn,tp_box,fp_box\n")
min_scores = []
min_scores.append(.7)
min_scores.append(.75)
min_scores.append(.80)
min_scores.append(.85)
min_scores.append(.90)
min_scores.append(.95)
for min_score in min_scores:
    gt_obj_pres_pos = 0
    gt_obj_pres_neg = 0
    tp=0
    fp=0
    tn=0
    fn=0
    tp_box=0
    fp_box=0

    for cn in range(0,100):
    
        obj_f_num = test_file_names[cn]
        obj_f_num = obj_f_num[:10]
        #print(fn)
        annot_edited_path = os.path.join(image_prediction_folder_edited, (obj_f_num+'.xml'))    
        tree = ElementTree.parse(annot_edited_path)
        print(annot_edited_path)
        #print(tree)
        # get the root of the document
        root = tree.getroot()
        # extract each bounding box
   
        #print(fn_image)
        box_num=0
        boxes_correct={}
        obj_pres_pred = 0
        for obj in root.findall('./object'):
            score = obj.find('score').text
            score = float(score)
            if score>=min_score:
                correct = obj.find('correct').text
                if(correct is None):
                    correct="0"
                if(correct==""):
                    correct="0"
                correct=int(correct)
                if(correct==0):
                    fp_box=fp_box+1
                if(correct==1):
                    tp_box=tp_box+1
                #if one is correct, it's deemed found in the picture.
                obj_pres_pred = 1
                #obj_pres_pred or correct
                #print(correct,obj_pres_pred)
        object_present = root.find('./object_present').text
        if(object_present=="1"):
            gt_obj_pres_pos = gt_obj_pres_pos + 1
            if(obj_pres_pred==1):
                tp=tp+1
            if(obj_pres_pred==0):
                fn=fn+1
        if(object_present=="0"):
            gt_obj_pres_neg = gt_obj_pres_neg + 1
            if(obj_pres_pred==1):
                fp=fp+1
            if(obj_pres_pred==0):
                tn=tn+1
    print("gt_obj_pres_pos", gt_obj_pres_pos)
    print("gt_obj_pres_neg", gt_obj_pres_neg)
    #results.write("cfg_name,gt_obj_pres,gt_obj_not_pres,TP,FP,TN,FN\n")
    results.write(str(min_score) + "," + str(gt_obj_pres_pos) + "," + str(gt_obj_pres_neg) + "," + str(tp) + "," + str(fp) + "," + str(tn) + "," + str(fn) + "," + str(tp_box) + "," + str(fp_box) + "\n")

results.close()

cfg_name: cfg20200826T2315
/home/student/charcoal_hearth_hill/predictions/cfg20200826T2315/unknown/edited/013pan1323.xml
/home/student/charcoal_hearth_hill/predictions/cfg20200826T2315/unknown/edited/073pas4017.xml
/home/student/charcoal_hearth_hill/predictions/cfg20200826T2315/unknown/edited/180pan1009.xml
/home/student/charcoal_hearth_hill/predictions/cfg20200826T2315/unknown/edited/128pas0807.xml
/home/student/charcoal_hearth_hill/predictions/cfg20200826T2315/unknown/edited/104pas0506.xml
/home/student/charcoal_hearth_hill/predictions/cfg20200826T2315/unknown/edited/170pas1509.xml
/home/student/charcoal_hearth_hill/predictions/cfg20200826T2315/unknown/edited/073pas4911.xml
/home/student/charcoal_hearth_hill/predictions/cfg20200826T2315/unknown/edited/123pan1206.xml
/home/student/charcoal_hearth_hill/predictions/cfg20200826T2315/unknown/edited/031pan0301.xml
/home/student/charcoal_hearth_hill/predictions/cfg20200826T2315/unknown/edited/221pan1311.xml
/home/student/charcoal_hearth_hil